In [170]:
DAY = "17"

In [171]:
import os
with open(f'session.secret','r') as f:
    SESSION = f"session={f.read()}"

INPUT_FILE = f"{DAY}.txt"

In [189]:
# Manual INPUT
INPUT ="""Register A: 2024
Register B: 0
Register C: 0

Program: 0,3,5,4,3,0"""

In [200]:
# Automatic INPUT
import os.path
if not os.path.isfile(INPUT_FILE):
    !curl https://adventofcode.com/2024/day/{DAY}/input --cookie {SESSION} > {INPUT_FILE}
with open(INPUT_FILE,'r') as f:
    INPUT = f.read().strip()

## Usefull import

In [236]:
import numpy as np
import pandas as pd
import networkx as nx
import copy as cp
import re

In [237]:
DIR = {(0,1):'>',(1,0):'v',(0,-1):'<',(-1,0):'^'}
DIRd = {v:k for k,v in DIR.items()}

## Solution

In [238]:
def combo(x):
    if 0<x<=3:
        return x
    if x==4:
        return A
    if x==5:
        return B
    if x==6:
        return C
    raise Exception("Invalid")

In [239]:
output = []
def adv(x) :
    global A,B,C,output
    A = A>>combo(x)
def bxl(x):
    global A,B,C,output
    B= B^x
def bst(x):
    global A,B,C,output
    B=combo(x)%8
def jnz(x):
    global A,B,C,output
    if A==0:
        return None
    return x
def bxc(x):
    global A,B,C,output
    B=B^C
def out(x):
    global A,B,C,output
    output.append(combo(x)%8)
def bdv(x):
    global A,B,C,output
    B = A>>combo(x)
def cdv(x):
    global A,B,C,output
    C = A>>combo(x)

In [240]:
OP= {
    0: adv,
    1:bxl,
    2:bst,
    3:jnz,
    4:bxc,
    5:out,
    6:bdv,
    7:cdv
}

In [267]:
output.clear()
A = int(re.findall(r"Register A:\s+([0-9]+)",INPUT)[0])
B = int(re.findall(r"Register B:\s+([0-9]+)",INPUT)[0])
C = int(re.findall(r"Register C:\s+([0-9]+)",INPUT)[0])
program = [int(x) for x in re.findall(r"Program:\s+([0-9,]+)",INPUT)[0].split(',')]
ptr=0
jump=None
i = 0
while 0<=ptr<len(program):
    jump = OP[program[ptr]](program[ptr+1])
    if jump is not None:
        ptr=jump
        jump=None
    else:
        ptr+=2
    
print(','.join([str(o) for o in output]))

2,4,1,7,7,5,1,7,0,3,4,1,5,5,3,0


In [250]:
def is_correct(A,result):
    # Note: this only work for my program, so different input might use different check
    # But I think solution remain the same: 
    #      A is shifted by 3 bit each loop and loop stop when A=0
    #      You can determine the last output of your program based on the first 3 bits of A
    #      Using the firsts 3 bits of A you can check the output before with all next 3 bits
    # To make is work:
    #    check for one loop if the output is the same as result for a given A
    B= A%8
    B = B^7
    C = A>>B
    B = B^7
    A = A>>3
    B = B^C
    return B%8==result

In [264]:
def solve(A,results):
    if len(results) == 0:
        return [A]
    all_solution = []
    for a in range(8):
        possible_A = (A<<3)|a
        if is_correct(possible_A,results[-1]):
            all_solution+=solve(possible_A,results[:-1])
    return all_solution
        

In [266]:
min(solve(0,program))

265601188299675